In [221]:
#part2: bert feature-base
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings
import jieba.posseg as pseg
warnings.filterwarnings('ignore')

In [222]:
# 读取数据
train = pd.read_csv('./train_samples.csv')
test = pd.read_csv('./test_samples.csv')
# train = pd.read_csv('./train.tsv.txt', delimiter='\t', header=None)



In [223]:
train_set = train[:1]

print("Train set shape:", train_set.shape)
train_set

Train set shape: (1, 13)


,id,name,text,中文名,外文名,别名,拼音,国籍,本名,作者,类型,书名,作品名称
0,6625,工业文明,敫桂英是京剧折子戏《打神告庙》中的人物，敫桂英曾救助落难书生王魁，两人在海神庙山盟海誓，王魁...,1,0,0,0,0,0,0,0,0,0


In [224]:
# train_set['id'].value_counts()
# train_set['id']
train_set['text']

0    敫桂英是京剧折子戏《打神告庙》中的人物，敫桂英曾救助落难书生王魁，两人在海神庙山盟海誓，王魁...
Name: text, dtype: object

In [225]:
model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-chinese')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)



In [226]:
train_tokenized = train_set['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(train_tokenized)
trainWordList = []
testWordList = []
# COMMENT = 'text'
# for item in train[COMMENT]:
#     # print(item)
#     words1 = [word.word for word in pseg.cut(item)]
#     trainWordList.append(','.join(words1))
#     # print(trainWordList)
# for item in test[COMMENT]:
#     # print(item)
#     words1 = [word.word for word in pseg.cut(item)]
#     # print()
#     testWordList.append(','.join(words1))
# train_tokenized = trainWordList

0    [101, 100, 100, 1941, 100, 1755, 100, 100, 181...
Name: text, dtype: object


In [227]:
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i)) for i in train_tokenized.values])
print("train set shape:",train_padded.shape)

train set shape: (1, 512)


In [228]:
print(len(train_padded))
train_attention_mask = np.where(train_padded != 0, 1, 0)
print(train_attention_mask)

1
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [229]:
# 训练集
train_input_ids = torch.tensor(train_padded).long()
train_attention_mask = torch.tensor(train_attention_mask).long()
print(train_attention_mask)
print(train_input_ids)
with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1

In [230]:
train_last_hidden_states[0].size()
train_last_hidden_states

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3480,  0.3887, -0.0124,  ..., -0.5743,  1.0525, -0.1379],
         [-0.0545,  0.7581,  0.6177,  ..., -0.2175,  1.4272, -0.2338],
         [-0.2729,  0.6778,  0.9668,  ..., -0.4568,  1.0704, -0.4259],
         ...,
         [-0.0667,  0.7076,  0.5926,  ..., -0.8695,  0.6562, -0.9724],
         [-0.0875,  0.6998,  0.8099,  ..., -0.8338,  0.6209, -1.0542],
         [-0.7274,  0.4321, -0.2352,  ..., -0.0594,  0.2842, -0.2291]]]), pooler_output=tensor([[-0.7922, -0.4048, -0.5280,  0.5772,  0.2542, -0.1371,  0.4785,  0.3117,
         -0.1506, -0.9998,  0.2009,  0.6711,  0.9752,  0.1854,  0.8672, -0.4214,
          0.2507, -0.4038,  0.3670, -0.0241,  0.6396,  0.9990,  0.3088,  0.3638,
          0.5287,  0.5230, -0.5437,  0.8810,  0.9217,  0.5781, -0.5439,  0.3265,
         -0.9783, -0.2660, -0.7821, -0.9790,  0.4470, -0.5202, -0.1861, -0.2739,
         -0.7857,  0.2938,  0.9990,  0.0042, -0.1419, -0.3685, -0.9999,  0.

In [231]:
train_features = train_last_hidden_states[0][:,0,:].numpy()
train_labels = train_set[1]

KeyError: 1

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(train_features, train_labels)

In [ ]:
print(train_features)
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

[[-1.1526822   0.36264995 -1.1256177  ...  1.8206166   0.31102756
  -0.36791456]
 [-0.17445564  0.7405766  -1.1302775  ...  0.34932986 -0.29706264
  -0.18439297]
 [-0.72042805  0.45488027  0.07139891 ...  0.3666889   0.25715974
   0.09777701]
 ...
 [-0.39177954  0.56023866 -0.05291298 ...  0.54079616 -0.23185462
  -0.0999588 ]
 [-0.3408751   0.89259213 -0.4946783  ...  0.8575598  -0.01450866
  -0.43705964]
 [-0.57745713  0.80726665 -0.46764064 ...  0.97600496 -0.26437128
  -0.5727735 ]]


LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.49333333333333335

In [ ]:
#part 2 - bert fine-tuned
import torch
from torch import nn
from torch import optim
import transformers as tfs
import math

class BertClassificationModel(nn.Module):
    def __init__(self):
        super(BertClassificationModel, self).__init__()   
        model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, 'bert-base-uncased')         
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.bert = model_class.from_pretrained(pretrained_weights)
        self.dense = nn.Linear(768, 2)  #bert默认的隐藏单元数是768， 输出单元是2，表示二分类
        
    def forward(self, batch_sentences):
        #print(batch_sentences[4])
        batch_tokenized = self.tokenizer.batch_encode_plus(batch_sentences, add_special_tokens=True,
                                max_len=66, pad_to_max_length=True)      #tokenize、add special token、pad
        input_ids = torch.tensor(batch_tokenized['input_ids'])
        attention_mask = torch.tensor(batch_tokenized['attention_mask'])
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        bert_cls_hidden_state = bert_output[0][:,0,:]       #提取[CLS]对应的隐藏状态
        linear_output = self.dense(bert_cls_hidden_state)
        return linear_output

In [ ]:
sentences = train_set[0].values
targets = train_set[1].values
train_inputs, test_inputs, train_targets, test_targets = train_test_split(sentences, targets)

batch_size = 64
batch_count = int(len(train_inputs) / batch_size)
batch_train_inputs, batch_train_targets = [], []
for i in range(batch_count):
    batch_train_inputs.append(train_inputs[i*batch_size : (i+1)*batch_size])
    batch_train_targets.append(train_targets[i*batch_size : (i+1)*batch_size])

In [ ]:
#train the model
epochs = 3
lr = 0.01
print_every_batch = 10
bert_classifier_model = BertClassificationModel()
optimizer = optim.SGD(bert_classifier_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print_avg_loss = 0
    for i in range(batch_count):
        inputs = batch_train_inputs[i]
        labels = torch.tensor(batch_train_targets[i])
        optimizer.zero_grad()
        outputs = bert_classifier_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        print_avg_loss += loss.item()
        if i % print_every_batch == (print_every_batch-1):
            print("Batch: %d, Loss: %.4f" % ((i+1), print_avg_loss/print_every_batch))
            print_avg_loss = 0
        

Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.


In [ ]:
# eval the trained model
total = len(test_inputs)
hit = 0
with torch.no_grad():
    for i in range(total):
        outputs = bert_classifier_model([test_inputs[i]])
        _, predicted = torch.max(outputs, 1)
        if predicted == test_targets[i]:
            hit += 1

print("Accuracy: %.2f%%" % (hit / total * 100))

Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.
Keyword arguments {'max_len': 66} not recognized.


Accuracy: 54.67%
